In [98]:
import os
import os.path as osp
import torch_geometric as pyg
import pandas as pd
import torch
import collections
import seaborn as sns
import torchvision
from typing import List

In [3]:
graph_dir = "../graphs/resnet34/cifar10"
graph_files = [osp.join(graph_dir, i) for i in os.listdir(graph_dir)]
test_graph = graph_files[0]

In [223]:
def pairingLayers(layernames:List[str])->List[str]:
    pairs = []
    for i in range(1, len(layernames)):
        cur = layernames[i]
        prev = layernames[i-1]
        if cur == 'fc' or prev == 'fc':
            continue
        if 'downsample' not in cur:
            pairs.append([prev, cur])
            if 'downsample' in prev:
                pairs.append([pairs[-3][-1], cur])
        else:
            components = cur.split('.')
            sublayer = int(components[1])
            if sublayer == 0:
                for j in range(i-1, -1, -1):
                    prev_layer_comp = layernames[j].split('.')
                    if prev_layer_comp[0] != components[0]:
                        pairs.append([layernames[j], cur])
                        break   
            else:
                for j in range(i-1, -1, -1):
                    prev_layer_comp = layernames[j].split('.')
                    if int(prev_layer_comp[1]) < sublayer:
                        pairs.append([layernames[j], cur])
                        break   
            
    return pairs

In [224]:
def CopelandScores(l1:dict, l2:dict, l2inear=False)->List[float]:
    l1_out = l1['dim_out']
    l2_in  = l2['dim_in']
    k_size = l2_in // l1_out
    l1deg = pyg.utils.degree(l1['graph'].edge_index[0])
    l2deg = pyg.utils.degree(l2['graph'].edge_index[0])
    ####
    in_deg  = l1deg[-l1_out::]
    out_deg = l2deg[0:l2_in].reshape(l1_out, k_size)
    ###
    in_deg_norm = in_deg / (l1deg[0:l1['dim_in']] != 0).sum()
    out_deg_norm = out_deg / (l2deg[l2['dim_in']::]!=0).sum()
    ###
    mask = in_deg != 0
    ###
    out_deg_m = out_deg_norm[mask]
    in_deg_m = in_deg_norm[mask]
    throughput = out_deg_m * in_deg_m.tile(k_size).view(k_size, in_deg_m.size(0)).T
    return throughput.mean()
    

In [225]:
def ChannelOverlapCoef(l1:dict, in_channels:int)->float:
    k_size = l1['dim_in'] // in_channels
    channel_coefs = []
    for c in range(in_channels):
        overlap_nodes = None
        min_graph = float("inf")
        for k in range(k_size):
            node = c * k_size + k
            mask = l1['graph'].edge_index[0] == node
            tgt_nodes = set(l1['graph'].edge_index[1, mask].tolist())
            min_graph = min(min_graph, mask.float().sum().item())
            if overlap_nodes is None:
                overlap_nodes = tgt_nodes
            else:
                overlap_nodes = overlap_nodes.intersection(tgt_nodes)
        if min_graph == 0:
            continue
        coef = len(overlap_nodes) / min_graph
        channel_coefs.append(coef)
    return sum(channel_coefs)/len(channel_coefs)

In [232]:
graph = torch.load(test_graph)
conv1 = graph['layer1.0.conv1']
conv2 = graph['layer1.0.conv2']
thput = CopelandScores(graph["layer4.2.conv2"], graph["fc"] )

RuntimeError: shape '[512, 0]' is invalid for input of size 10

In [233]:
graph['fc']

{'idx_in_start': 64531,
 'idx_out_start': 64541,
 'dim_in': 10,
 'dim_out': 512,
 'sparsity': 0.5,
 'graph': Data(edge_index=[2, 5120], edge_attr=[5120, 1], num_nodes=522),
 'ram_scores': ((0.6177238297953693, 1.1729952312659913, 45.73686),
  (3.1750934212958315, 18.782793695806056, 4.595269))}

In [227]:
coef = ChannelOverlapCoef(conv2, conv1['dim_out'])

In [228]:
pairs = pairingLayers(list(graph.keys()))

In [229]:
pairs

[['conv1', 'layer1.0.conv1'],
 ['layer1.0.conv1', 'layer1.0.conv2'],
 ['layer1.0.conv2', 'layer1.1.conv1'],
 ['layer1.1.conv1', 'layer1.1.conv2'],
 ['layer1.1.conv2', 'layer1.2.conv1'],
 ['layer1.2.conv1', 'layer1.2.conv2'],
 ['layer1.2.conv2', 'layer2.0.conv1'],
 ['layer2.0.conv1', 'layer2.0.conv2'],
 ['layer1.2.conv2', 'layer2.0.downsample.0'],
 ['layer2.0.downsample.0', 'layer2.1.conv1'],
 ['layer2.0.conv2', 'layer2.1.conv1'],
 ['layer2.1.conv1', 'layer2.1.conv2'],
 ['layer2.1.conv2', 'layer2.2.conv1'],
 ['layer2.2.conv1', 'layer2.2.conv2'],
 ['layer2.2.conv2', 'layer2.3.conv1'],
 ['layer2.3.conv1', 'layer2.3.conv2'],
 ['layer2.3.conv2', 'layer3.0.conv1'],
 ['layer3.0.conv1', 'layer3.0.conv2'],
 ['layer2.3.conv2', 'layer3.0.downsample.0'],
 ['layer3.0.downsample.0', 'layer3.1.conv1'],
 ['layer3.0.conv2', 'layer3.1.conv1'],
 ['layer3.1.conv1', 'layer3.1.conv2'],
 ['layer3.1.conv2', 'layer3.2.conv1'],
 ['layer3.2.conv1', 'layer3.2.conv2'],
 ['layer3.2.conv2', 'layer3.3.conv1'],
 ['lay

In [ ]:
### graph = torch.load(test_graph)
conv1 = graph['layer1.0.conv1']
conv2 = graph['layer1.0.conv2']
conv1_out = conv1['dim_out']
conv2_in = conv2['dim_in']
kernel_size = conv2_in // conv1_out
conv1_degree = pyg.utils.degree(conv1['graph'].edge_index[0])
conv2_degree = pyg.utils.degree(conv2['graph'].edge_index[0])
in_degree = conv1_degree[conv1['dim_in']::]
out_degree = conv2_degree[0:conv2_in].reshape(conv1_out, kernel_size)

in_degree_norm = in_degree / (conv1_degree[0:conv1['dim_in']]!=0).sum()
out_degree_norm = out_degree / (conv2_degree[conv2['dim_in']::] != 0).sum()

compatibility = ((in_degree != 0) & (out_degree.mean(dim=1) != 0)).float().sum()
compatibility /= ((in_degree != 0) | (out_degree.mean(dim=1) != 0)).float().sum()





In [166]:
m = conv1["graph"].edge_index[0] == 0
conv1['graph'].edge_index[1, m].unqiue()

AttributeError: 'Tensor' object has no attribute 'toset'

In [19]:
conv2

{'idx_in_start': 723,
 'idx_out_start': 1299,
 'dim_in': 576,
 'dim_out': 64,
 'sparsity': 0.3809136284722222,
 'graph': Data(edge_index=[2, 28084], edge_attr=[28084, 1], num_nodes=640),
 'ram_scores': ((0.2094198735529998, 0.26276230557066715, 89.220634),
  (3.5243808478978886, 20.392490381547425, 5.3018456))}

In [159]:
thput.mean()

tensor(0.2730)

In [77]:
out_degree_norm.mean(dim=-1)

tensor([0.5074, 0.0000, 0.6148, 0.4537, 0.6407, 0.3907, 0.5259, 0.0000, 0.4889,
        0.5519, 0.4704, 0.4407, 0.0000, 0.0000, 0.3519, 0.4741, 0.5389, 0.4944,
        0.4500, 0.0000, 0.5574, 0.0000, 0.0000, 0.6111, 0.4759, 0.4537, 0.7870,
        0.5907, 0.6056, 0.5870, 0.1778, 0.0000, 0.6222, 0.0000, 0.4370, 0.4019,
        0.5074, 0.0000, 0.5648, 0.4241, 0.6111, 0.0000, 0.0000, 0.0000, 0.6667,
        0.6148, 0.5167, 0.5815, 0.5333, 0.4222, 0.5630, 0.0000, 0.5667, 0.6037,
        0.4944, 0.6537, 0.6352, 0.4389, 0.4204, 0.5130, 0.5333, 0.4352, 0.5167,
        0.4852])

In [70]:
conv2_degree[conv2['dim_in']::]

tensor([215., 219., 262., 254., 172., 278., 290., 270., 207., 238., 215., 175.,
        209., 128., 259., 181., 321., 198., 199., 247., 211., 216.,   0.,   0.,
        264., 257.,   0., 260., 272., 172.,   0., 260., 275., 238., 313., 248.,
        269., 269., 203., 229., 117., 188., 275., 279., 240., 250., 286., 275.,
        302., 265., 246., 243., 255., 260., 299., 197., 207., 226.,  94., 213.,
        191., 290., 196., 155.])

In [141]:
(in_degree_rel == 0).sum()

tensor(14)

In [134]:
in_degree_rel.tile(9).view(9, 64).T

tensor([[0.3785, 0.3785, 0.3785, 0.3785, 0.3785, 0.3785, 0.3785, 0.3785, 0.3785],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3958, 0.3958, 0.3958, 0.3958, 0.3958, 0.3958, 0.3958, 0.3958, 0.3958],
        [0.2899, 0.2899, 0.2899, 0.2899, 0.2899, 0.2899, 0.2899, 0.2899, 0.2899],
        [0.4392, 0.4392, 0.4392, 0.4392, 0.4392, 0.4392, 0.4392, 0.4392, 0.4392],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
        [0.3663, 0.3663, 0.3663, 0.3663, 0.3663, 0.3663, 0.3663, 0.3663, 0.3663],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3559, 0.3559, 0.3559, 0.3559, 0.3559, 0.3559, 0.3559, 0.3559, 0.3559],
        [0.3889, 0.3889, 0.3889, 0.3889, 0.3889, 0.3889, 0.3889, 0.3889, 0.3889],
        [0.3837, 0.3837, 0.3837, 0.3837, 0.3837, 0.3837, 0.3837, 0.3837, 0.3837],
        [0.3316, 0.3316, 0.3316, 0.3316, 0.3316, 0.3316, 0.3316, 0.3316, 0.3316],
        [0.0000,

In [130]:
out_degree_rel.shape

torch.Size([64, 9])

In [95]:
compatibility

tensor(1.)

In [86]:
(in_degree != 0).float()

tensor([1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
        1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1.,
        1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [88]:
(out_degree.mean(dim=1) != 0).float()

tensor([1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
        1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1.,
        1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [216]:
list(graph.keys())

['conv1',
 'layer1.0.conv1',
 'layer1.0.conv2',
 'layer1.1.conv1',
 'layer1.1.conv2',
 'layer1.2.conv1',
 'layer1.2.conv2',
 'layer2.0.conv1',
 'layer2.0.conv2',
 'layer2.0.downsample.0',
 'layer2.1.conv1',
 'layer2.1.conv2',
 'layer2.2.conv1',
 'layer2.2.conv2',
 'layer2.3.conv1',
 'layer2.3.conv2',
 'layer3.0.conv1',
 'layer3.0.conv2',
 'layer3.0.downsample.0',
 'layer3.1.conv1',
 'layer3.1.conv2',
 'layer3.2.conv1',
 'layer3.2.conv2',
 'layer3.3.conv1',
 'layer3.3.conv2',
 'layer3.4.conv1',
 'layer3.4.conv2',
 'layer3.5.conv1',
 'layer3.5.conv2',
 'layer4.0.conv1',
 'layer4.0.conv2',
 'layer4.0.downsample.0',
 'layer4.1.conv1',
 'layer4.1.conv2',
 'layer4.2.conv1',
 'layer4.2.conv2',
 'fc']

In [23]:
model = torchvision.models.resnet34(num_classes=10)


In [24]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  